## Upload Books

In [21]:
import requests

url = "http://127.0.0.1:8080"
book_name = "Art of Coaching Script.json"
book_path = r"E:\ro2ya_dev\LARA\books\Art of Coaching Script.json"
upload_type = "application/vnd.openxmlformats-officedocument.wordprocessingml.document"

payload = {}
files=[
  ('file',(book_name, open(book_path,'rb'), upload_type))
]
headers = {}

response = requests.request("POST", url, headers=headers, data=payload, files=files)

print(response.text)

{"filename":"Art of Coaching Script.json","location":"books/Art of Coaching Script.json"}


## Test Stream Text

In [22]:
import requests
import json

stream_url = 'http://localhost:8080/text'
data = {
    "message": "ما الذي يتكلم عنه الكتاب أو الكورس وما آثاره الإيجابية؟"
}  

headers = {'Content-Type': 'application/json'}

with requests.post(stream_url, data=json.dumps(data),headers=headers, stream=True) as response:
    if response.status_code == 200:
        for chunk in response:
            print(chunk.decode('utf-8'), end="")
    else:
        print("Failed to connect to the streaming service")


الكورس بيتحدث عن فن التوجيه والتدريب "coaching" وكيفية تطبيقه في بيئة العمل. ومن ثم يسلط الضوء على الآثار الإيجابية لتبني هذه الثقافة في الشركات، مثل تحسين أداء الموظفين والمديرين، زيادة الإنتاجية، تحسين العلاقات الفريقية وتطوير مهارات الموظفين.

## Test Stream Audio

In [23]:
import requests
import json
from audio import stream

stream_url = 'http://localhost:8080/audio'
data = {
    "message": "what is your name"
}

headers = {'Content-Type': 'application/json'}

with requests.post(stream_url, data=json.dumps(data),headers=headers, stream=True) as response:
    if response.status_code == 200:
        stream(response)
    else:
        print("Failed to connect to the streaming service")

print("Streaming completed.")

Streaming completed.


# Stream Text and Audio

In [24]:
import asyncio
import websockets
import json
from audio import stream, astream_with_text
import subprocess
import base64

async def send_data_via_websocket():
    uri = "ws://localhost:8080/socket_audio"
    data = {
        "message": "شكرا لارا"
    }
    
    try:
        async with websockets.connect(uri) as websocket:
            await websocket.send(json.dumps(data))
            await astream_with_text(websocket)
            
    except Exception as e:
        print(f"An error occurred: {e}")

loop = asyncio.get_event_loop()
if loop.is_running():
    asyncio.ensure_future(send_data_via_websocket())
else:
    loop.run_until_complete(send_data_via_websocket())

على الرحب والسعة! أنا في الخدمة دايمًا.An error occurred: received 1012 (service restart); then sent 1012 (service restart)
